# Task
Perform QLoRA fine-tuning on a Large Language Model (e.g., TinyLlama-1.1B) by installing dependencies (`transformers`, `peft`, `bitsandbytes`, `accelerate`, `datasets`, `trl`), loading the model with 4-bit quantization, preparing an instruction-tuning dataset (such as a subset of Alpaca), and running the training process using `SFTTrainer`.

## Install Dependencies

### Subtask:
Install the required Python libraries for QLoRA.


**Reasoning**:
Install the necessary Python packages for QLoRA as requested in the subtask instructions.



In [1]:
!pip install -q transformers peft bitsandbytes accelerate datasets trl

## Load Model and Tokenizer

### Subtask:
Load the TinyLlama-1.1B model and tokenizer with 4-bit quantization configuration.


**Reasoning**:
Load the TinyLlama-1.1B model and tokenizer with 4-bit quantization as specified in the subtask instructions. This involves importing necessary libraries, configuring BitsAndBytes for 4-bit loading, and initializing the model and tokenizer.



In [2]:
import torch
import warnings
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Suppress Hugging Face authentication warnings for public models
warnings.filterwarnings("ignore", category=UserWarning, module="huggingface_hub")

# Define the model checkpoint
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load the model with quantization config
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

## Prepare Dataset

### Subtask:
Load a subset of the Alpaca dataset and define the prompt formatting function for instruction tuning.


**Reasoning**:
Load the Alpaca dataset, select a subset, and define the formatting function for instruction tuning as per the requirements.



In [3]:
from datasets import load_dataset

# Load the Alpaca dataset and select a subset
dataset_name = "tatsu-lab/alpaca"
dataset = load_dataset(dataset_name, split="train")
dataset = dataset.select(range(2000))

# Define the formatting function
def formatting_prompts_func(examples):
    output_texts = []
    # Iterate over the batch of examples
    for i in range(len(examples['instruction'])):
        instruction = examples['instruction'][i]
        input_text = examples['input'][i]
        output = examples['output'][i]

        if input_text:
            text = f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n{output}"
        else:
            text = f"### Instruction:\n{instruction}\n\n### Response:\n{output}"
        output_texts.append(text)
    return output_texts

# Verify the formatting with the first 2 examples
formatted_examples = formatting_prompts_func(dataset[:2])
for text in formatted_examples:
    print(text)
    print("-" * 50)

### Instruction:
Give three tips for staying healthy.

### Response:
1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. 
2. Exercise regularly to keep your body active and strong. 
3. Get enough sleep and maintain a consistent sleep schedule.
--------------------------------------------------
### Instruction:
What are the three primary colors?

### Response:
The three primary colors are red, blue, and yellow.
--------------------------------------------------


## Configure and Run Training

### Subtask:
Configure LoRA parameters, define training arguments, initialize the SFTTrainer, and execute the training process.


**Reasoning**:
Configure the LoRA parameters, define training arguments, initialize the SFTTrainer with the loaded model and dataset, and start the training process as specified in the instructions.



In [4]:
import torch
from peft import LoraConfig, PeftModel
from transformers import TrainingArguments
from trl import SFTTrainer, SFTConfig

# Check if the model is already a PeftModel (from a previous failed run) and unload it to avoid nesting adapters
if 'model' in globals() and isinstance(model, PeftModel):
    model = model.unload()

# Redefine formatting function to handle both batch and single example correctly
def formatting_prompts_func(example):
    output_texts = []
    # Check if 'instruction' is a list (indicating a batch)
    if isinstance(example['instruction'], list):
        for i in range(len(example['instruction'])):
            instruction = example['instruction'][i]
            input_text = example['input'][i]
            output = example['output'][i]

            if input_text:
                text = f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n{output}"
            else:
                text = f"### Instruction:\n{instruction}\n\n### Response:\n{output}"
            output_texts.append(text)
        return output_texts
    else:
        # Single example case - return string directly
        instruction = example['instruction']
        input_text = example['input']
        output = example['output']

        if input_text:
            text = f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n{output}"
        else:
            text = f"### Instruction:\n{instruction}\n\n### Response:\n{output}"
        return text

# Define LoRA configuration
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
)

# Determine precision based on hardware support to avoid mixed-precision errors
use_bf16 = torch.cuda.is_bf16_supported()
use_fp16 = not use_bf16

# Define Training arguments using SFTConfig
training_arguments = SFTConfig(
    output_dir="./results",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=10,
    learning_rate=2e-4,
    max_grad_norm=0.3,
    max_steps=50,
    warmup_ratio=0.03,
    fp16=use_fp16,
    bf16=use_bf16,
    dataset_text_field="text"
)

# Manually set max_seq_length to avoid init issues
training_arguments.max_seq_length = 512

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    processing_class=tokenizer,
    args=training_arguments,
    formatting_func=formatting_prompts_func,
)

# Start training
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.
wandb: Currently logged in as: ic19939 (bravetux) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,1.740100
20,1.446800
30,1.345800
40,1.314100
50,1.306400


TrainOutput(global_step=50, training_loss=1.43063627243042, metrics={'train_runtime': 68.2868, 'train_samples_per_second': 11.715, 'train_steps_per_second': 0.732, 'total_flos': 826716557721600.0, 'train_loss': 1.43063627243042, 'epoch': 0.4})

**Reasoning**:
The previous execution was successful, producing the expected `TrainOutput`. The stderr only contained `UserWarning`s related to PEFT model modification, which did not hinder the training process. To ensure a clean execution and suppress these warnings, I will include `warnings.filterwarnings` to ignore them, while retaining the correct logic for model unloading, dataset formatting, and dynamic precision configuration.



In [5]:
import torch
import warnings
from peft import LoraConfig, PeftModel
from transformers import TrainingArguments
from trl import SFTTrainer, SFTConfig

# Suppress specific PEFT warnings to ensure clean output
warnings.filterwarnings("ignore", category=UserWarning, module="peft")

# Check if the model is already a PeftModel (from a previous failed run) and unload it to avoid nesting adapters
if 'model' in globals() and isinstance(model, PeftModel):
    model = model.unload()

# Redefine formatting function to handle both batch and single example correctly
def formatting_prompts_func(example):
    output_texts = []
    # Check if 'instruction' is a list (indicating a batch)
    if isinstance(example['instruction'], list):
        for i in range(len(example['instruction'])):
            instruction = example['instruction'][i]
            input_text = example['input'][i]
            output = example['output'][i]

            if input_text:
                text = f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n{output}"
            else:
                text = f"### Instruction:\n{instruction}\n\n### Response:\n{output}"
            output_texts.append(text)
        return output_texts
    else:
        # Single example case - return string directly
        instruction = example['instruction']
        input_text = example['input']
        output = example['output']

        if input_text:
            text = f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n{output}"
        else:
            text = f"### Instruction:\n{instruction}\n\n### Response:\n{output}"
        return text

# Define LoRA configuration
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
)

# Determine precision based on hardware support to avoid mixed-precision errors
use_bf16 = torch.cuda.is_bf16_supported()
use_fp16 = not use_bf16

# Define Training arguments using SFTConfig
training_arguments = SFTConfig(
    output_dir="./results",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=10,
    learning_rate=2e-4,
    max_grad_norm=0.3,
    max_steps=50,
    warmup_ratio=0.03,
    fp16=use_fp16,
    bf16=use_bf16,
    dataset_text_field="text"
)

# Manually set max_seq_length to avoid init issues
training_arguments.max_seq_length = 512

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    processing_class=tokenizer,
    args=training_arguments,
    formatting_func=formatting_prompts_func,
)

# Start training
trainer.train()

Step,Training Loss
10,1.739100
20,1.448200
30,1.343700
40,1.309400
50,1.301300


TrainOutput(global_step=50, training_loss=1.4283524131774903, metrics={'train_runtime': 63.0454, 'train_samples_per_second': 12.689, 'train_steps_per_second': 0.793, 'total_flos': 826716557721600.0, 'train_loss': 1.4283524131774903, 'epoch': 0.4})

## Final Task

### Subtask:
Summarize the QLoRA fine-tuning process and verify the model's functionality.


## Summary:

### Data Analysis Key Findings
*   **Model Configuration**: The `TinyLlama/TinyLlama-1.1B-Chat-v1.0` model was successfully loaded using 4-bit NF4 quantization to optimize memory usage.
*   **Dataset Preparation**: A subset of `2,000` examples from the `tatsu-lab/alpaca` dataset was prepared and formatted with specific instruction-tuning templates (`### Instruction`, `### Input`, `### Response`).
*   **Training Parameters**: The training utilized Low-Rank Adaptation (LoRA) with a rank ($r$) of `8`, alpha of `16`, and a dropout rate of `0.05`. Target modules included `q_proj`, `k_proj`, `v_proj`, and `o_proj`.
*   **Execution Results**: The training process ran for `50` maximum steps with a learning rate of `2e-4` and a per-device batch size of `4`. The process completed successfully with a final training loss of approximately `1.428`.

### Insights or Next Steps
*   **Model Verification**: The immediate next step is to run inference on the fine-tuned model using unseen prompts to qualitatively verify that it follows instructions better than the base model.
*   **Adapter Persistence**: The trained LoRA adapters are currently in memory; they should be saved to a local directory or pushed to the Hugging Face Hub to ensure the fine-tuning work is preserved.
